# Load Sequence Dataset Dengan Custom Function

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical

def load_sequence_dataset(base_dir, img_size=(224, 224), seq_len=10, class_names=None):
    X, y = [], []

    if class_names is None:
        class_names = sorted(os.listdir(base_dir))
    class_to_idx = {name: i for i, name in enumerate(class_names)}

    for class_name in class_names:
        class_path = os.path.join(base_dir, class_name)
        for seq_folder in os.listdir(class_path):
            frames = sorted(os.listdir(os.path.join(class_path, seq_folder)))[:seq_len]
            sequence = []

            for frame in frames:
                img = cv2.imread(os.path.join(class_path, seq_folder, frame))
                img = cv2.resize(img, img_size)
                img = img.astype('float32') / 255.0
                sequence.append(img)

            if len(sequence) == seq_len:
                X.append(sequence)
                y.append(class_to_idx[class_name])

    return np.array(X), to_categorical(y, num_classes=len(class_names))


# Membangun Model CNN + LSTM + Dropout

In [ ]:
from tensorflow.keras import layers, models, Input
import tensorflow as tf

def build_cnn_lstm_model(seq_len=10, input_shape=(224, 224, 3), num_classes=6):
    input_layer = Input(shape=(seq_len, *input_shape))

    base_cnn = tf.keras.applications.ResNet50(
        include_top=False,
        weights='imagenet',
        pooling='avg',
        input_shape=input_shape
    )
    base_cnn.trainable = False

    x = layers.TimeDistributed(base_cnn)(input_layer)
    x = layers.LSTM(128, return_sequences=True)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.LSTM(128)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    output = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


# Training Model

In [ ]:
X, y = load_sequence_dataset("dataset", img_size=(224, 224), seq_len=10)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = build_cnn_lstm_model(seq_len=10, input_shape=(224, 224, 3), num_classes=6)
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=25, batch_size=8)